In [19]:
import pandas as pd
import string
import re

import pos_tagger

# demo verisi

In [25]:
lyrics = []
lyrics.append(input())
df_dataset = pd.DataFrame()
df_dataset["sarki_sozu"] = lyrics

# Verilerin Temizlenmesi

In [21]:
def clean_text(text):
    # Satır boşluklarının kaldırılması
    text = text.replace("\n\n", "\n")

    # Satır Sonlarındaki Boşlukların Kaldırılması
    lines = text.split("\n")
    temp_text = ""
    for line in lines:
        temp_text += line.rstrip() + "\n"
    text = temp_text.rstrip("\n")

    # Noktalama İşaretlerinin Kaldırılması
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Küçük Harf Dönüşümü
    text = text.lower()

    return text

df_dataset["sarki_sozu"] = [clean_text(lyrics) for lyrics in df_dataset.loc[:,"sarki_sozu"]]

##### Eski Türkçe kelime oranlarının bulunması

In [22]:
my_file = open("../raw_data/eski_turkce_kelimeler.txt", "r")
old_turkish_words = my_file.read()
old_turkish_words = old_turkish_words.replace('\n', ' ').split(" ")
my_file.close()

def calculate_ratio(row):
    lyrics = re.split('\n| ', row['sarki_sozu'])
    matched_words = [word for word in lyrics if word in old_turkish_words]
    ratio = round(len(matched_words) / len(lyrics),2)
    return ratio

df_dataset['eski_turkce_kelime_orani'] = df_dataset.apply(calculate_ratio, axis=1)

##### Pos Tag oranlarının eklenmesi

In [23]:
verb_rate_list = []
adj_rate_list = []
adv_rate_list = []
conj_rate_list = []
interj_rate_list = []
ques_rate_list = []
unk_rate_list = []
for lyrics in df_dataset.loc[:, "sarki_sozu"]:
    pos_tags = pos_tagger.get_pos_tags(lyrics)
    verb_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Verb.name) / len(pos_tags), 2))
    adj_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Adj.name) / len(pos_tags), 2))
    adv_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Adv.name) / len(pos_tags), 2))
    conj_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Conj.name) / len(pos_tags), 2))
    interj_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Interj.name) / len(pos_tags), 2))
    ques_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Ques.name) / len(pos_tags), 2))
    unk_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Unk.name) / len(pos_tags), 2))

df_dataset["fiil_orani"] = verb_rate_list
df_dataset["sifat_orani"] = adj_rate_list
df_dataset["zarf_orani"] = adv_rate_list
df_dataset["baglac_orani"] = conj_rate_list
df_dataset["unlem_orani"] = interj_rate_list
df_dataset["soru_orani"] = ques_rate_list
df_dataset["bilinmeyen_orani"] = unk_rate_list

In [24]:
df_dataset.to_csv("../dataset/demo_dataset.csv", index=False)